In [1]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import datasets
import numpy as np
import pandas as pd

### Градиентный бустинг (20 баллов)


В текущем ДЗ вам предлагается дописать свою реализацию градиентного бустинга над решающими деревьями. Основной костяк класса прописан, вам нужно заполнить строчки с TODO.

In [2]:
GLOBAL_RANDOM_STATE = 42
# Параметрами с которыми вы хотите обучать деревья
TREE_PARAMS_DICT = {"max_depth": 3, "random_state": GLOBAL_RANDOM_STATE}
# Параметр tau (learning_rate) для вашего GB
TAU = 0.1

In [3]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters, tau):
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        self.estimators = []

    def fit(self, X_data, y_data):
        curr_pred = np.mean(y_data)
        self.first_pred = curr_pred
        for iter_num in range(self.iters):
            # Нужно посчитать антиградиент функции потерь по предсказниям в точке curr_pred
            residuals = 2 * (y_data - curr_pred)
            # Нужно обучить DecisionTreeRegressor предсказывать антиградиент
            algo = DecisionTreeRegressor(**self.tree_params_dict)
            algo.fit(X_data, residuals)
            self.estimators.append(algo)
            # Обновите предсказания в каждой точке
            curr_pred += self.tau * algo.predict(X_data)
        return self

    def predict(self, X_data):
        # Предсказание на данных
        res = self.first_pred
        for estimator in self.estimators:
            # нужно сложить все предсказания деревьев с весом self.tau
            res += self.tau * estimator.predict(X_data)
        return res

# Проверка

Подгрузим данные.

In [4]:
import warnings
warnings.filterwarnings("ignore")

boston = datasets.load_boston()

In [5]:
data = boston['data']
target = boston['target']

In [6]:
from sklearn.ensemble import GradientBoostingRegressor

In [7]:
gbr = GradientBoostingRegressor(max_depth=3, random_state=GLOBAL_RANDOM_STATE)

Качество реализации GBR из sklearn.

In [9]:
np.mean(cross_val_score(gbr, data, target, cv=3, scoring='neg_mean_squared_error'))

-27.641784915305465

In [8]:
algo = SimpleGB(tree_params_dict=TREE_PARAMS_DICT, iters=100, tau=TAU)

Качество вашего алгоритма может отличаться, но не более чем на 5%. При худшем качестве, баллы не выставляются. Вы можете поменять параметры, или улучшить основной код алгоритма, чтобы добиться лучшего качества. Это будет оценено при проверке.

In [10]:
np.mean(cross_val_score(algo, data, target, cv=3, scoring='neg_mean_squared_error'))

-28.194152230762786